In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np

project_root = os.path.abspath('..')
sys.path.append(project_root)

from sleeprnn.helpers import reader, misc
from sleeprnn.detection.feeder_dataset import FeederDataset
from sleeprnn.detection import metrics
from sleeprnn.common.optimal_thresholds import OPTIMAL_THR_FOR_CKPT_DICT
from sleeprnn.common import constants, pkeys, viz

RESULTS_PATH = os.path.join(project_root, 'results')

# Old implementation

In [ ]:
ckpt_folder = '20210411_bsf_update_exp1'
# You may specify certain runs within that ckpt_folder in grid_folder_list.
# If None then all runs are returned
grid_folder_list = None
# You may specify certain seeds within the experiment to plot.
# If None, then all available seeds are plotted
selected_seeds = None

dataset_name = constants.MASS_SS_NAME
train_fraction = 0.75
fs = 200
which_expert = 1
task_mode = constants.N2_RECORD
set_list = [constants.VAL_SUBSET]
verbose = False

# Plot settings
legend_fontsize = 9  # 7
show_splits_id = True
show_subject_id = True
show_grid = True
show_desired_attractor = True
show_mean = True
show_quadrants = True
skip_subjects = []  # [11, 14, 19]
iou_to_show = 0.2
marker_size = 7
marker_alpha = 1.0

# ----------------------

# Identifier
result_id = '%s-%s-E%d-%s' % (
    dataset_name.split('_')[0].upper(),
    dataset_name.split('_')[1].upper(),
    which_expert,
    task_mode.upper())

# Define paths
full_ckpt_folder = '%s_%s_train_%s' % (ckpt_folder, task_mode, dataset_name)
if grid_folder_list is None:
    grid_folder_list = os.listdir(os.path.join(
        RESULTS_PATH, 'predictions_%s' % dataset_name, full_ckpt_folder))
grid_folder_list.sort()
print("Checkpoint folder:\n%s" % full_ckpt_folder)
print("Considered settings:")
pprint(grid_folder_list)
save_dir = os.path.join(
    RESULTS_PATH,
    'auto_pr_figs',
    full_ckpt_folder)
# Find available seeds
if selected_seeds is None:
    available_seed_folders = os.listdir(os.path.abspath(os.path.join(
        RESULTS_PATH,
        'predictions_%s' % dataset_name,
        full_ckpt_folder, grid_folder_list[0]
    )))
    seeds_to_show = [int(f[4:]) for f in available_seed_folders]
    seeds_to_show.sort()
else:
    seeds_to_show = selected_seeds
print("Seeds to plot: %s" % seeds_to_show)
print("")

# Load data and predictions
dataset = reader.load_dataset(dataset_name, params={pkeys.FS: fs}, verbose=verbose)
ids_dict = {constants.ALL_TRAIN_SUBSET: dataset.train_ids}
ids_dict.update(misc.get_splits_dict(dataset, seeds_to_show, use_test_set=False, train_fraction=train_fraction))
predictions_dict = {}
for grid_folder in grid_folder_list:
    full_grid_path = os.path.join(full_ckpt_folder, grid_folder)
    predictions_dict[grid_folder] = reader.read_prediction_with_seeds(
        full_grid_path, dataset_name, task_mode, seeds_to_show, set_list=set_list, parent_dataset=dataset,
        verbose=verbose
    )

# Plot
print("\nProcessing %s" % full_ckpt_folder, flush=True)
# if save_figs:
#     os.makedirs(save_dir, exist_ok=True)
#     print("Saving directory: %s" % save_dir)
color_dict = {
    constants.TRAIN_SUBSET: {
        i: viz.GREY_COLORS[4] for i in range(4)},
    constants.VAL_SUBSET: {
        0: viz.PALETTE[constants.RED],
        1: viz.PALETTE[constants.BLUE],
        2: viz.PALETTE[constants.GREEN],
        3: viz.PALETTE[constants.DARK],
        4: viz.PALETTE[constants.CYAN],
        5: viz.PALETTE[constants.PURPLE],
        6: viz.PALETTE[constants.GREY],
    }
}
axis_markers = np.arange(0, 1.1, 0.1)
for grid_folder in grid_folder_list:
    full_grid_path = os.path.join(full_ckpt_folder, grid_folder)
    fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=100)
    tmp_all_recall = []
    tmp_all_precision = []
    tmp_all_mean_iou = []
    for seed_id in seeds_to_show:
        # ---------------- Compute performance
        pre_vs_iou_subject_dict = {}
        rec_vs_iou_subject_dict = {}
        mean_iou_subject_dict = {}
        for set_name in set_list:
            # Prepare expert labels
            data_inference = FeederDataset(dataset, ids_dict[seed_id][set_name], task_mode, which_expert)
            this_ids = data_inference.get_ids()
            this_events_list = data_inference.get_stamps()
            # Prepare model predictions
            prediction_obj = predictions_dict[grid_folder][seed_id][set_name]
            prediction_obj.set_probability_threshold(OPTIMAL_THR_FOR_CKPT_DICT[full_grid_path][seed_id])
            this_detections_list = prediction_obj.get_stamps()
            for i, single_id in enumerate(this_ids):
                single_events = this_events_list[i]
                single_detections = this_detections_list[i]
                this_iou_matching, _ = metrics.matching(single_events, single_detections)
                this_mean_iou = np.mean(this_iou_matching[this_iou_matching > 0])
                this_precision = metrics.metric_vs_iou(
                    single_events, single_detections, [iou_to_show], metric_name=constants.PRECISION,
                    iou_matching=this_iou_matching)
                this_recall = metrics.metric_vs_iou(
                    single_events, single_detections, [iou_to_show], metric_name=constants.RECALL,
                    iou_matching=this_iou_matching)
                pre_vs_iou_subject_dict[single_id] = this_precision[0]
                rec_vs_iou_subject_dict[single_id] = this_recall[0]
                mean_iou_subject_dict[single_id] = this_mean_iou

        # -------------------- P L O T ----------------------
        for set_name in set_list:
            for i, single_id in enumerate(ids_dict[seed_id][set_name]):
                if single_id in skip_subjects:
                    continue
                this_rec = rec_vs_iou_subject_dict[single_id]
                this_pre = pre_vs_iou_subject_dict[single_id]
                this_iou = mean_iou_subject_dict[single_id]
                tmp_all_recall.append(this_rec)
                tmp_all_precision.append(this_pre)
                tmp_all_mean_iou.append(this_iou)
                label = 'Split %d' % seed_id if i == 0 else None
                if show_splits_id:
                    color = color_dict[set_name][seed_id]
                else:
                    color = viz.PALETTE['blue']
                ax.plot(
                    this_rec, this_pre, color=color, alpha=marker_alpha, markeredgewidth=0.0,
                    marker='o', markersize=marker_size, label=label, linestyle='None')
                if show_subject_id:
                    if isinstance(single_id, str):
                        single_id_to_show = int(single_id[0] + single_id[3:])
                        subject_id_fontsize = 3
                    else:
                        single_id_to_show = single_id
                        subject_id_fontsize = 4
                    ax.annotate(
                        single_id_to_show, (this_rec, this_pre),
                        horizontalalignment="center", verticalalignment="center",
                        fontsize=subject_id_fontsize, color="w")
    tmp_all_precision = np.array(tmp_all_precision)
    tmp_all_recall = np.array(tmp_all_recall)
    tmp_all_f1_score = 2 * tmp_all_precision * tmp_all_recall / (tmp_all_precision + tmp_all_recall)
    print("Precision %1.1f \u00B1 %4.1f -- Recall %1.1f \u00B1 %4.1f -- F1-score %1.1f \u00B1 %1.1f -- mIoU %1.1f \u00B1 %1.1f for %s" % (
        100 * np.mean(tmp_all_precision), 100 * np.std(tmp_all_precision),
        100 * np.mean(tmp_all_recall), 100 * np.std(tmp_all_recall),
        100 * np.mean(tmp_all_f1_score), 100 * np.std(tmp_all_f1_score),
        100 * np.mean(tmp_all_mean_iou), 100 * np.std(tmp_all_mean_iou),
        grid_folder
    ))
    perf_str = "F1: %1.1f\u00B1%1.1f, IoU: %1.1f\u00B1%1.1f\nP: %1.1f\u00B1%1.1f, R: %1.1f\u00B1%1.1f" % (
        100 * np.mean(tmp_all_f1_score), 100 * np.std(tmp_all_f1_score),
        100 * np.mean(tmp_all_mean_iou), 100 * np.std(tmp_all_mean_iou),
        100 * np.mean(tmp_all_precision), 100 * np.std(tmp_all_precision),
        100 * np.mean(tmp_all_recall), 100 * np.std(tmp_all_recall),
    )
    ax.plot([0, 1], [0, 1], zorder=1, linewidth=1, color=viz.GREY_COLORS[4])
    ax.set_title('%s\nValidation, %s\n%s' % (grid_folder, result_id, perf_str), fontsize=9)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_yticks(axis_markers)
    ax.set_xticks(axis_markers)
    if show_grid:
        ax.set_xticks(np.arange(0, 1, 0.1), minor=True)
        ax.set_yticks(np.arange(0, 1, 0.1), minor=True)
        ax.grid(which="minor")
    if show_quadrants:
        ax.axhline(0.5, color=viz.GREY_COLORS[5], linewidth=2)
        ax.axvline(0.5, color=viz.GREY_COLORS[5], linewidth=2)
    if show_desired_attractor:
        ax.fill_between([0.80, 0.9], 0.8, 0.9, facecolor=viz.GREY_COLORS[2], zorder=1)
    if show_mean:
        ax.plot(
            np.mean(tmp_all_recall), np.mean(tmp_all_precision),
            marker='o', markersize=marker_size / 2, linestyle="None",
            color=viz.GREY_COLORS[6]
        )
    ax.tick_params(labelsize=8.5)
    ax.set_ylabel('Precision (IoU>%1.1f)' % iou_to_show, fontsize=9)
    ax.set_xlabel('Recall (IoU>%1.1f)' % iou_to_show, fontsize=9)
    ax.set_aspect('equal')
    if show_splits_id:
        ax.legend(loc='lower left', fontsize=legend_fontsize)
    plt.tight_layout()
    # if save_figs:
    #     fname = os.path.join(save_dir, "pr_seeds_%s.png" % grid_folder)
    #     plt.savefig(fname, dpi=viz.DPI, bbox_inches="tight", pad_inches=0.01)
    plt.show()
    plt.close()


# New implementation - By fold

In [ ]:
def format_precision_recall_plot(
    ax,
    axis_markers=None, 
    show_quadrants=True,
    show_grid=True, 
    minor_axis_markers=None
):
    if axis_markers is None:
        axis_markers = np.arange(0, 1 + 0.001, 0.1)
    if minor_axis_markers is None:
        minor_axis_markers = axis_markers
    # Diagonal
    ax.plot([0, 1], [0, 1], zorder=1, linewidth=1, color=viz.GREY_COLORS[4])
    # Square basis
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_aspect('equal')
    # Ticks
    ax.set_yticks(axis_markers)
    ax.set_xticks(axis_markers)
    ax.set_xticks(minor_axis_markers, minor=True)
    ax.set_yticks(minor_axis_markers, minor=True)
    if show_grid:    
        ax.grid(which="minor")
    if show_quadrants:
        ax.axhline(0.5, color=viz.GREY_COLORS[5], linewidth=2)
        ax.axvline(0.5, color=viz.GREY_COLORS[5], linewidth=2)


def get_fold_colors():
    color_dict = {
        0: viz.PALETTE[constants.RED],
        1: viz.PALETTE[constants.BLUE],
        2: viz.PALETTE[constants.GREEN],
        3: viz.PALETTE[constants.DARK],
        5: viz.PALETTE[constants.PURPLE],
        4: viz.PALETTE[constants.CYAN],
        6: viz.PALETTE[constants.GREY],
    }
    return color_dict


def get_performance_string(outputs):
    perf_str = "F1: %1.1f\u00B1%1.1f, IoU: %1.1f\u00B1%1.1f\nP: %1.1f\u00B1%1.1f, R: %1.1f\u00B1%1.1f" % (
        100 * np.mean(outputs['f1']), 100 * np.std(outputs['f1']),
        100 * np.mean(outputs['miou']), 100 * np.std(outputs['miou']),
        100 * np.mean(outputs['prec']), 100 * np.std(outputs['prec']),
        100 * np.mean(outputs['rec']), 100 * np.std(outputs['rec']),
    )
    return perf_str

In [ ]:
dataset_name = constants.MASS_SS_NAME
which_expert = 1
task_mode = constants.N2_RECORD
dataset_params = {pkeys.FS: 200}
load_dataset_from_ckpt = True
dataset = reader.load_dataset(
    dataset_name, params=dataset_params, load_checkpoint=load_dataset_from_ckpt, verbose=False)

In [ ]:
ckpt_folder_prefix = '20210411_bsf_update_exp1'
# You may specify certain runs within that ckpt_folder in grid_folder_list.
# If None then all runs are returned
grid_folder_list = None
# You may specify certain folds within the experiment to plot.
# If None, then all available folds are used
selected_folds = None

# Data settings
dataset_name = constants.MASS_SS_NAME
which_expert = 1
task_mode = constants.N2_RECORD
dataset_params = {pkeys.FS: 200}
load_dataset_from_ckpt = True

# Evaluation settings
evaluation_set = constants.VAL_SUBSET
average_mode = constants.MACRO_AVERAGE
iou_threshold_report = 0.2

# Plot settings
title_fontsize = 9
general_fontsize = 9
marker_size = 7
marker_alpha = 1.0
fold_monocolor = False
show_fold_id = True
show_grid = True
show_mean = True
show_quadrants = True
axis_markers = np.arange(0, 1 + 0.001, 0.2)
minor_axis_markers = np.arange(0, 1 + 0.001, 0.1)

# -----------------------------------------------------------
# -----------------------------------------------------------
color_dict = get_fold_colors()
dataset = reader.load_dataset(
    dataset_name, params=dataset_params, load_checkpoint=load_dataset_from_ckpt, verbose=False)
ckpt_folder = '%s_%s_train_%s' % (ckpt_folder_prefix, task_mode, dataset_name)
if grid_folder_list is None:
    experiment_path = os.path.join(RESULTS_PATH, 'predictions_%s' % dataset_name, ckpt_folder)
    grid_folder_list = os.listdir(experiment_path)
    grid_folder_list.sort()
print('Grid settings to be used from %s:' % ckpt_folder)
pprint(grid_folder_list)
predictions_dict = {}
for grid_folder in grid_folder_list:
    grid_folder_complete = os.path.join(ckpt_folder, grid_folder)
    predictions_dict[grid_folder] = reader.read_predictions_crossval(
        grid_folder_complete, dataset, task_mode)
if selected_folds is None:
    selected_folds = list(predictions_dict[grid_folder_list[0]].keys())
    selected_folds.sort()
print("Folds to plot: %s" % selected_folds)
print("")
result_id = '%s-%s-E%d-%s' % (
    dataset_name.split('_')[0].upper(),
    dataset_name.split('_')[1].upper(),
    which_expert,
    task_mode.upper())
metric_vs_iou_fn_dict = {
    constants.MACRO_AVERAGE: metrics.metric_vs_iou_macro_average,
    constants.MICRO_AVERAGE: metrics.metric_vs_iou_micro_average}
for grid_folder in grid_folder_list:
    fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=viz.DPI)
    opt_thr_list = OPTIMAL_THR_FOR_CKPT_DICT[os.path.join(ckpt_folder, grid_folder)]
    outputs = {'f1': [], 'miou': [], 'prec': [], 'rec': []}
    for k in selected_folds:
        # Retrieve relevant data
        eval_predictions = predictions_dict[grid_folder][k][evaluation_set]
        subject_ids = eval_predictions.get_ids()
        feed_d = FeederDataset(dataset, subject_ids, task_mode, which_expert=which_expert)
        events_list = feed_d.get_stamps()
        eval_predictions.set_probability_threshold(opt_thr_list[k])
        detections_list = eval_predictions.get_stamps()
        # Compute performance
        iou_matching_list, _ = metrics.matching_with_list(events_list, detections_list)
        f1_score = metric_vs_iou_fn_dict[average_mode](
            events_list, detections_list, [iou_threshold_report], 
            iou_matching_list=iou_matching_list, metric_name=constants.F1_SCORE)
        recall = metric_vs_iou_fn_dict[average_mode](
            events_list, detections_list, [iou_threshold_report], 
            iou_matching_list=iou_matching_list, metric_name=constants.RECALL)
        precision = metric_vs_iou_fn_dict[average_mode](
            events_list, detections_list, [iou_threshold_report], 
            iou_matching_list=iou_matching_list, metric_name=constants.PRECISION)
        nonzero_iou_list = [iou_matching[iou_matching > 0] for iou_matching in iou_matching_list]
        if average_mode == constants.MACRO_AVERAGE:
            miou_list = [np.mean(nonzero_iou) for nonzero_iou in nonzero_iou_list]
            miou = np.mean(miou_list)
        elif average_mode == constants.MICRO_AVERAGE:
            miou = np.concatenate(nonzero_iou_list).mean()
        else:
            raise ValueError("Average mode %s invalid" % average_mode)
        outputs['f1'].append(f1_score[0])
        outputs['prec'].append(precision[0])
        outputs['rec'].append(recall[0])
        outputs['miou'].append(miou)
        # Plot
        color = viz.PALETTE['blue'] if fold_monocolor else color_dict[k]
        ax.plot(
            recall[0], precision[0], 
            color=color, linestyle='None', alpha=marker_alpha, 
            markeredgewidth=0.0, marker='o', markersize=marker_size, 
            label='Fold %d' % k)
    if show_mean:
        ax.plot(
            np.mean(outputs['rec']), np.mean(outputs['prec']),
            marker='o', markersize=marker_size / 2, linestyle="None",
            color=viz.GREY_COLORS[6])
    perf_str = get_performance_string(outputs)
    eval_str = "%s-%s" % (average_mode.split("_")[0].upper(), evaluation_set.upper())
    ax.set_title(
        '%s\n%s, %s\n%s' % (
            grid_folder, eval_str, result_id, perf_str), 
        fontsize=title_fontsize)
    ax.tick_params(labelsize=general_fontsize)
    ax.set_xlabel('Recall (IoU>%1.1f)' % iou_threshold_report, fontsize=general_fontsize)
    ax.set_ylabel('Precision (IoU>%1.1f)' % iou_threshold_report, fontsize=general_fontsize)
    if show_fold_id:
        ax.legend(loc='lower left', fontsize=general_fontsize)
    format_precision_recall_plot(
        ax, axis_markers=axis_markers, show_quadrants=show_quadrants,
        show_grid=show_grid, minor_axis_markers=minor_axis_markers)
    plt.tight_layout()
    plt.show()

# New implementation - By subject

In [ ]:
ckpt_folder_prefix = '20210411_bsf_update_exp1'
# You may specify certain runs within that ckpt_folder in grid_folder_list.
# If None then all runs are returned
grid_folder_list = None
# You may specify certain folds within the experiment to plot.
# If None, then all available folds are used
selected_folds = None

# Data settings
dataset_name = constants.MASS_SS_NAME
which_expert = 1
task_mode = constants.N2_RECORD
dataset_params = {pkeys.FS: 200}
load_dataset_from_ckpt = True

# Evaluation settings
evaluation_set = constants.VAL_SUBSET
iou_threshold_report = 0.2

# Plot settings
title_fontsize = 9
general_fontsize = 9
marker_size = 7
marker_alpha = 0.6
show_subject_id = True
group_by_subject = False
subject_to_highlight = [14, 5]
fold_monocolor = True
show_fold_id = False
show_grid = False
show_mean = True
show_quadrants = False
axis_markers = np.arange(0, 1 + 0.001, 0.2)
minor_axis_markers = np.arange(0, 1 + 0.001, 0.1)

# -----------------------------------------------------------
# -----------------------------------------------------------
color_dict = get_fold_colors()
# dataset = reader.load_dataset(
#     dataset_name, params=dataset_params, load_checkpoint=load_dataset_from_ckpt, verbose=False)
ckpt_folder = '%s_%s_train_%s' % (ckpt_folder_prefix, task_mode, dataset_name)
if grid_folder_list is None:
    experiment_path = os.path.join(RESULTS_PATH, 'predictions_%s' % dataset_name, ckpt_folder)
    grid_folder_list = os.listdir(experiment_path)
    grid_folder_list.sort()
print('Grid settings to be used from %s:' % ckpt_folder)
pprint(grid_folder_list)
predictions_dict = {}
for grid_folder in grid_folder_list:
    grid_folder_complete = os.path.join(ckpt_folder, grid_folder)
    predictions_dict[grid_folder] = reader.read_predictions_crossval(
        grid_folder_complete, dataset, task_mode)
if selected_folds is None:
    selected_folds = list(predictions_dict[grid_folder_list[0]].keys())
    selected_folds.sort()
print("Folds to plot: %s" % selected_folds)
print("")
result_id = '%s-%s-E%d-%s' % (
    dataset_name.split('_')[0].upper(),
    dataset_name.split('_')[1].upper(),
    which_expert,
    task_mode.upper())
for grid_folder in grid_folder_list:
    fig, ax = plt.subplots(1, 1, figsize=(4, 4), dpi=viz.DPI)
    opt_thr_list = OPTIMAL_THR_FOR_CKPT_DICT[os.path.join(ckpt_folder, grid_folder)]
    outputs = {'f1': [], 'miou': [], 'prec': [], 'rec': [], 'fold': [], 'subjects': []}
    for k in selected_folds:
        # Retrieve relevant data
        eval_predictions = predictions_dict[grid_folder][k][evaluation_set]
        subject_ids = eval_predictions.get_ids()
        feed_d = FeederDataset(dataset, subject_ids, task_mode, which_expert=which_expert)
        events_list = feed_d.get_stamps()
        eval_predictions.set_probability_threshold(opt_thr_list[k])
        detections_list = eval_predictions.get_stamps()
        # Compute performance
        iou_matching_list, _ = metrics.matching_with_list(events_list, detections_list)
        f1_score = metrics.metric_vs_iou_macro_average(
            events_list, detections_list, [iou_threshold_report], 
            iou_matching_list=iou_matching_list, metric_name=constants.F1_SCORE, collapse_values=False)
        recall = metrics.metric_vs_iou_macro_average(
            events_list, detections_list, [iou_threshold_report], 
            iou_matching_list=iou_matching_list, metric_name=constants.RECALL, collapse_values=False)
        precision = metrics.metric_vs_iou_macro_average(
            events_list, detections_list, [iou_threshold_report], 
            iou_matching_list=iou_matching_list, metric_name=constants.PRECISION, collapse_values=False)
        nonzero_iou_list = [iou_matching[iou_matching > 0] for iou_matching in iou_matching_list]
        miou_list = [np.mean(nonzero_iou) for nonzero_iou in nonzero_iou_list]
        outputs['f1'].append(f1_score[:, 0])
        outputs['prec'].append(precision[:, 0])
        outputs['rec'].append(recall[:, 0])
        outputs['miou'].append(miou_list)
        outputs['subjects'].append(subject_ids)
        outputs['fold'].append([k] * len(subject_ids))
    if group_by_subject:
        subject_ids = np.unique(np.concatenate(outputs['subjects']))
        grouped_outputs = {
            s: {key: [] for key in outputs}
            for s in subject_ids}
        for k in selected_folds:
            for i, s in enumerate(outputs['subjects'][k]):
                for key in outputs.keys():
                    grouped_outputs[s][key].append(outputs[key][k][i])
        outputs = {key: [] for key in grouped_outputs[subject_ids[0]].keys()}
        for s in subject_ids:
            for key in grouped_outputs[s].keys():
                if key in ['fold', 'subjects']:
                    # Assign to the first fold in which it appears
                    outputs[key].append(grouped_outputs[s][key][0])
                else:
                    outputs[key].append(np.mean(grouped_outputs[s][key]))
    else:
        for key in outputs.keys():
            outputs[key] = np.concatenate(outputs[key])
    sorted_loc = np.argsort(outputs['fold'])
    for key in outputs.keys():
        outputs[key] = np.asarray(outputs[key])[sorted_loc]
    # Plot
    folds_shown = []
    for i in range(len(outputs['f1'])):
        subject_id = outputs['subjects'][i]
        k = outputs['fold'][i]
        if fold_monocolor:
            color = viz.PALETTE['green'] if subject_id in subject_to_highlight else viz.PALETTE['blue']
        else:
            color = color_dict[k]
        label = 'Fold %d' % k if k not in folds_shown else None
        folds_shown.append(k)
        ax.plot(
            outputs['rec'][i], outputs['prec'][i], 
            color=color, linestyle='None', alpha=marker_alpha, 
            markeredgewidth=0.0, marker='o', markersize=marker_size, 
            label=label)
        if show_subject_id:
            if isinstance(subject_id, str):
                single_id_to_show = int(subject_id[0] + subject_id[3:])
                subject_id_fontsize = 3
            else:
                single_id_to_show = subject_id
                subject_id_fontsize = 4
            ax.annotate(
                single_id_to_show, (outputs['rec'][i], outputs['prec'][i]),
                horizontalalignment="center", verticalalignment="center",
                fontsize=subject_id_fontsize, color="w")
    if show_mean:
        ax.plot(
            np.mean(outputs['rec']), np.mean(outputs['prec']),
            marker='o', markersize=marker_size / 2, linestyle="None",
            color=viz.GREY_COLORS[6])
    perf_str = get_performance_string(outputs)
    eval_str = "%s-%s" % ("SUBJECT", evaluation_set.upper())
    ax.set_title(
        '%s\n%s, %s\n%s' % (
            grid_folder, eval_str, result_id, perf_str), 
        fontsize=title_fontsize)
    ax.tick_params(labelsize=general_fontsize)
    ax.set_xlabel('Recall (IoU>%1.1f)' % iou_threshold_report, fontsize=general_fontsize)
    ax.set_ylabel('Precision (IoU>%1.1f)' % iou_threshold_report, fontsize=general_fontsize)
    if show_fold_id:
        ax.legend(loc='lower left', fontsize=general_fontsize)
    format_precision_recall_plot(
        ax, axis_markers=axis_markers, show_quadrants=show_quadrants,
        show_grid=show_grid, minor_axis_markers=minor_axis_markers)
    plt.tight_layout()
    plt.show()